In [34]:
import csv
import matplotlib.pyplot as plt
import nltk
import pandas
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
# import nltk
# nltk.download('stopwords')

pandas.set_option('display.max_rows', 1000)

# loading the dataset
dataset = pandas.read_csv("newDataset.csv",sep=',',usecols=['tweet','label'])

#print(type(dataset))
print(f"Number of tweets: {len(dataset)}")
display(dataset)

Number of tweets: 699


,tweet,label
0,Vaccini: il successo della Brexit premetterà a...,1
1,Aifa autorizza il vaccino AstraZeneca ma ne ra...,1
2,vaccini Mentre l'EU cerca (tra l'altro senza s...,0
3,Si andrebbe presumibilmente ben oltre l'anno. ...,2
4,"Oggi 30 gennaio , ho ricevuto la seconda dose ...",1
5,"In realtà, soprattutto nel caso di confronto c...",2
6,Quale sarebbe il vantaggio del nuovo vaccino t...,0
7,firewall76 nicolatosti1 ScaltritiLab rominafuo...,0
8,vaccino vaccini vaccinoCovid mammamia bambini ...,0
9,antonellaviol17 Dipende. Cosa succederà se avr...,1


In [35]:
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

italian_stemmer = SnowballStemmer('italian')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([italian_stemmer.stem(w) for w in analyzer(doc)])

In [36]:
# SIMPLE PIPELINE WITHOUT ANY TEST ON ngram_range AND CONSIDERING ONLY ACCURACY AS METRIC
# SE GLI ALTRI TEST VANNO BENE QUESTO SI PUO' ANCHE ELIMINARE

text_clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
folds =  10
#calculating accuracies in cross-validation
scores = cross_val_score(text_clf, dataset.tweet, dataset.label, cv=folds)
("Accuracy MultinomialNB : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#print(scores)

'Accuracy MultinomialNB : 0.51 (+/- 0.14)'

In [37]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average = 'weighted', zero_division = 0),
           'recall' : make_scorer(recall_score, average = 'macro'), 
           'f1_score' : make_scorer(f1_score, average = 'macro')
          }

def print_metrics(scores):
    print(f'accuracy on test set: {scores["test_accuracy"].mean():.3} +/- {scores["test_accuracy"].std()*2:.3}')
    print(f'precision on test set: {scores["test_precision"].mean():.3} +/- {scores["test_precision"].std()*2:.3}')
    print(f'recall on test set: {scores["test_recall"].mean():.3} +/- {scores["test_recall"].std()*2:.3}')
    print(f'f1-score on test set: {scores["test_f1_score"].mean():.3} +/- {scores["test_f1_score"].std()*2:.3}')

In [38]:
max_ngram = 2

folds = 10
print('NAIVE BAYES CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', MultinomialNB()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)

NAIVE BAYES CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.508 +/- 0.141
precision on test set: 0.567 +/- 0.131
recall on test set: 0.48 +/- 0.14
f1-score on test set: 0.475 +/- 0.162
------- ngram = (1, 2) -------
accuracy on test set: 0.532 +/- 0.155
precision on test set: 0.594 +/- 0.138
recall on test set: 0.499 +/- 0.155
f1-score on test set: 0.492 +/- 0.171
------- ngram = (2, 2) -------
accuracy on test set: 0.471 +/- 0.127
precision on test set: 0.51 +/- 0.138
recall on test set: 0.446 +/- 0.124
f1-score on test set: 0.445 +/- 0.131


In [39]:
print('DECISION TREE CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', tree.DecisionTreeClassifier()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)

DECISION TREE CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.423 +/- 0.114
precision on test set: 0.423 +/- 0.108
recall on test set: 0.418 +/- 0.105
f1-score on test set: 0.414 +/- 0.108
------- ngram = (1, 2) -------
accuracy on test set: 0.429 +/- 0.145
precision on test set: 0.429 +/- 0.144
recall on test set: 0.427 +/- 0.141
f1-score on test set: 0.423 +/- 0.141
------- ngram = (2, 2) -------
accuracy on test set: 0.39 +/- 0.112
precision on test set: 0.419 +/- 0.132
recall on test set: 0.396 +/- 0.102
f1-score on test set: 0.377 +/- 0.108


In [40]:
print('SVM CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', svm.LinearSVC()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring=scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)
            #print(scores)

SVM CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.522 +/- 0.112
precision on test set: 0.527 +/- 0.112
recall on test set: 0.515 +/- 0.103
f1-score on test set: 0.516 +/- 0.107
------- ngram = (1, 2) -------
accuracy on test set: 0.535 +/- 0.148
precision on test set: 0.539 +/- 0.147
recall on test set: 0.53 +/- 0.141
f1-score on test set: 0.53 +/- 0.147
------- ngram = (2, 2) -------
accuracy on test set: 0.449 +/- 0.134
precision on test set: 0.456 +/- 0.14
recall on test set: 0.438 +/- 0.129
f1-score on test set: 0.438 +/- 0.13


In [43]:
# code for confusion matrix
from sklearn.model_selection import cross_val_predict
text_clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (1,2))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
y_pred = cross_val_predict(text_clf, dataset.tweet, dataset.label, cv=folds)
#conf_mat = confusion_matrix(dataset.label, y_pred, normalize="true")
conf_mat = confusion_matrix(dataset.label, y_pred)
print(conf_mat)

[[0.2247191  0.52808989 0.24719101]
 [0.0233463  0.61478599 0.3618677 ]
 [0.01893939 0.3219697  0.65909091]]
